#### Imports

In [1]:
from scipy import spatial, sparse
from scipy.stats import chi2
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.externals import joblib 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os
import imp
import gzip
import copy
import nltk
import pickle
import scipy
import string
import gensim
import operator
import datetime

import numpy as np
import pandas as pd
import LDA_ELJST as lda
import matplotlib.pyplot as plt

In [2]:
import utils as my_utils
from sentiment import SentimentAnalysis

### Read Data

In [3]:
dataset_name = "amazon_musical"

In [1]:
dataset = pd.read_pickle("datasets/datadf_amazon_musical")

NameError: name 'pd' is not defined

In [5]:
dataset.head(2)

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,sentiment,reviewTime,summary,cleaned,text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]",1393545600,"Not much to write about here, but it does exac...",5.0,"02 28, 2014",good,"[much, write, doe, exactly, supposed, filter, ...",much write doe exactly supposed filter pop sou...
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",1363392000,The product does exactly as it should and is q...,5.0,"03 16, 2013",Jake,"[product, doe, exactly, quite, affordable, rea...",product doe exactly quite affordable realized ...


In [6]:
count_matrix, _, vocabulary, words = my_utils.processReviews(dataset['text'].values)

In [7]:
Counter(dataset['sentiment'])

Counter({1.0: 214, 2.0: 245, 3.0: 756, 4.0: 2050, 5.0: 6785})

In [8]:
barren = np.where(count_matrix.sum(1)==0)[0]

In [9]:
barren

array([], dtype=int64)

In [10]:
ratings = dataset['sentiment'].values

### Making Edge_Dict

In [11]:
dataset_name

'amazon_musical'

In [12]:
# docs_edges = pickle.load(open("resources/amazon_musical_bert_0.8.pickle","rb"))

In [13]:
# docs_edges = np.delete(docs_edges, barren).tolist()

## Run Model

In [14]:
maxiter = 20
lambda_param = 10.0
n_topics = 10
n_sentiment = 5

In [15]:
dataset_name

'amazon_musical'

In [ ]:
folder_name = str(datetime.datetime.now()) + "_" + dataset_name
os.mkdir("dumps/"+folder_name)

In [18]:
edge_list = ["resources/amazon_musical_bert_0.2.pickle",
             "resources/amazon_musical_bert_0.3.pickle",
             "resources/amazon_musical_bert_0.4.pickle",
             "resources/amazon_musical_bert_0.5.pickle",
             "resources/amazon_musical_fasttext_0.2.pickle",
             "resources/amazon_musical_fasttext_0.3.pickle",
             "resources/amazon_musical_fasttext_0.4.pickle",
             "resources/amazon_musical_fasttext_0.5.pickle",
             "resources/amazon_musical_glove_0.2.pickle",
             "resources/amazon_musical_glove_0.3.pickle",
             "resources/amazon_musical_glove_0.4.pickle",
             "resources/amazon_musical_glove_0.5.pickle"]

In [ ]:
import multiprocessing

In [ ]:
def multiprocessing_func(edge_loc):
    docs_edges = pickle.load(open(edge_loc,"rb"))
    sampler = lda.LdaSampler(count_matrix, ratings, docs_edges, words, vocabulary,
                         n_sentiment = n_sentiment, n_topics=n_topics, lambda_param=lambda_param)

    print "Running Sampler...", edge_loc
    sampler.run(maxiter=maxiter)
    joblib.dump(sampler, "dumps/" + folder_name + "/sampler_" + edge_loc)

In [ ]:
pool = multiprocessing.Pool()

In [ ]:
pool.map(multiprocessing_func, topics_grid)

In [19]:
docs_edges = pickle.load(open(edge_list[3],"rb"))

In [21]:
sampler = lda.LdaSampler(count_matrix[:100], ratings[:100], docs_edges[:100], words, vocabulary,
                         n_sentiment = n_sentiment, n_topics=4, lambda_param=lambda_param)

In [31]:
sampler.phi()[0,:,0].argsort()[-K:]

array([1971,  233])